## using parallelize

In [ ]:
tasks = []
for manufacturer in manufacturers:
    for gender in genders:
        for speech_task in speech_tasks:
            for lower_threshold in lower_thresholds:
                for higher_threshold in higher_thresholds:            
                    tasks = tasks + [(manufacturer, gender, speech_task, lower_threshold, higher_threshold)]
                    
tasksRDD = sc.parallelize(tasks, numSlices = len(tasks))

cfc = tasksRDD.map(lambda alpha: compute_ML_grid_for_a_cohort(alpha[0], alpha[1], alpha[2], alpha[3], alpha[4]))
cfc.cache()
cfc.count()

## using python threads for parallelization

In [ ]:
from threading import Thread
num_cohorts = len(manufacturers) * len(genders) * len(speech_tasks) * len(lower_thresholds) * len(higher_thresholds)
tasks = [None] * num_cohorts
results = [None] * num_cohorts
  
i = 0
for manufacturer in manufacturers:
    for gender in genders:
        for speech_task in speech_tasks:
            for lower_threshold in lower_thresholds:
                for higher_threshold in higher_thresholds:            
                    tasks[i] = Thread(target=compute_ML_grid_for_a_cohort, 
                                      args=(manufacturer, gender, speech_task, lower_threshold, higher_threshold, results, i))
                    tasks[i].start()
                    i += 1
                    
for i in range(len(tasks)):
    tasks[i].join()
    print 'results[' +str(i) + '] = ' + str(results[i])